In [13]:
import pandas as pd

# Load CSV
csv_file = "../onlinedeliverydata.csv"
df = pd.read_csv(csv_file)

# Clean column names: remove spaces, replace invalid characters
df.columns = [col.strip().replace(" ", "_").replace("-", "_").replace("/", "_") for col in df.columns]

# Replace 'nil' or 'NIL' with None
df = df.replace(["nil", "NIL"], None)

# Preview the data
print("Data shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df.head())


Data shape: (388, 55)
Columns: ['Age', 'Gender', 'Marital_Status', 'Occupation', 'Monthly_Income', 'Educational_Qualifications', 'Family_size', 'latitude', 'longitude', 'Pin_code', 'Medium_(P1)', 'Medium_(P2)', 'Meal(P1)', 'Meal(P2)', 'Perference(P1)', 'Perference(P2)', 'Ease_and_convenient', 'Time_saving', 'More_restaurant_choices', 'Easy_Payment_option', 'More_Offers_and_Discount', 'Good_Food_quality', 'Good_Tracking_system', 'Self_Cooking', 'Health_Concern', 'Late_Delivery', 'Poor_Hygiene', 'Bad_past_experience', 'Unavailability', 'Unaffordable', 'Long_delivery_time', 'Delay_of_delivery_person_getting_assigned', 'Delay_of_delivery_person_picking_up_food', 'Wrong_order_delivered', 'Missing_item', 'Order_placed_by_mistake', 'Influence_of_time', 'Order_Time', 'Maximum_wait_time', 'Residence_in_busy_location', 'Google_Maps_Accuracy', 'Good_Road_Condition', 'Low_quantity_low_time', 'Delivery_person_ability', 'Influence_of_rating', 'Less_Delivery_time', 'High_Quality_of_package', 'Number_

In [14]:
import sys
import os

# Add src folder to Python path
sys.path.append(os.path.join(os.getcwd(), "..", "src"))

from db_config import create_connection

# Connect to MySQL
connection = create_connection()
if connection is None:
    raise Exception("Failed to connect to MySQL")

cursor = connection.cursor()

# Create table with cleaned column names
columns = df.columns
column_types = ", ".join([f"`{col}` VARCHAR(255)" for col in columns])
create_table_query = f"""
CREATE TABLE IF NOT EXISTS delivery_preferences (
    id INT AUTO_INCREMENT PRIMARY KEY,
    {column_types}
);
"""

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully")
except Exception as e:
    print(f"Error creating table: {e}")
finally:
    cursor.close()
    connection.close()

Connected to MYSQL database
Table created successfully


In [15]:
import sys
import os
import pandas as pd

# Add src folder to Python path
sys.path.append(os.path.join(os.getcwd(), "..", "src"))

from db_config import create_connection

# Connect to MySQL
connection = create_connection()
if connection is None:
    raise Exception("Failed to connect to MySQL")

cursor = connection.cursor()

# Replace NaN with None for MySQL
df_clean = df.where(pd.notnull(df), None)

# Insert data row by row
columns = df_clean.columns
for i, row in df_clean.iterrows():
    placeholders = ", ".join(["%s"] * len(row))
    columns_str = ", ".join([f"`{col}`" for col in columns])
    insert_query = f"INSERT INTO delivery_preferences ({columns_str}) VALUES ({placeholders})"
    cursor.execute(insert_query, tuple(row))

connection.commit()
print(f"{len(df_clean)} rows inserted successfully")

cursor.close()
connection.close()

Connected to MYSQL database
388 rows inserted successfully


In [16]:
import sys
import os
import pandas as pd

# Add src folder to Python path
sys.path.append(os.path.join(os.getcwd(), "..", "src"))

from db_config import create_connection

# Connect to MySQL
connection = create_connection()
if connection is None:
    raise Exception("Failed to connect to MySQL")

cursor = connection.cursor()

# Select the correct database
database_name = "onlinefood_delivery"
cursor.execute(f"USE {database_name}")

# Check if the table exists
table_name = "delivery_preferences"
cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
result = cursor.fetchone()

if result:
    # Table exists, fetch first 5 rows
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
    rows = cursor.fetchall()
    
    # Fetch column names
    column_names = [i[0] for i in cursor.description]
    
    # Display as pandas DataFrame
    df_preview = pd.DataFrame(rows, columns=column_names)
    print(df_preview)
else:
    print(f"Table '{table_name}' does not exist in database '{database_name}'.")

cursor.close()
connection.close()

Connected to MYSQL database
    id Age  Gender Marital_Status Occupation  Monthly_Income  \
0  128  20  Female         Single    Student       No Income   
1  129  24  Female         Single    Student  Below Rs.10000   
2  130  22    Male         Single    Student  Below Rs.10000   
3  131  22  Female         Single    Student       No Income   
4  132  22    Male         Single    Student  Below Rs.10000   

  Educational_Qualifications Family_size latitude longitude  ...  \
0              Post Graduate           4  12.9766   77.5993  ...   
1                   Graduate           3   12.977   77.5773  ...   
2              Post Graduate           3  12.9551   77.6593  ...   
3                   Graduate           6  12.9473   77.5616  ...   
4              Post Graduate           4   12.985   77.5533  ...   

     Less_Delivery_time High_Quality_of_package       Number_of_calls  \
0  Moderately Important    Moderately Important  Moderately Important   
1        Very Important         